In [1]:
data_dir = 'dms_data'
import os
import pandas as pd
import numpy as np
import itertools
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.linear_model import LinearRegression

2023-09-15 10:30:20.377095: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 10:30:23.138496: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 10:30:23.168245: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 10:30:29.528104: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

def transform_to_graph_data(X, y):
    # Convert dataframe to tensor
    x = torch.tensor(X.values, dtype=torch.float)
    
    # Create edges by connecting consecutive nodes (timestamps)
    edge_index = torch.tensor([list(range(X.shape[0]-1)), list(range(1, X.shape[0]))], dtype=torch.long)
    
    # Convert target to tensor
    y = torch.tensor(y.values, dtype=torch.float).view(-1, 1)
    
    # Create a PyTorch Geometric data object
    data = Data(x=x, edge_index=edge_index, y=y)
    return data

class SimpleGNN(torch.nn.Module):
    def __init__(self, num_features):
        super(SimpleGNN, self).__init__()
        self.conv1 = GCNConv(num_features, 64)
        self.conv2 = GCNConv(64, 32)
        self.fc = torch.nn.Linear(32, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # 1st Graph Convolution
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        
        # 2nd Graph Convolution
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        
        # Fully Connected Layer
        x = self.fc(x)
        return x

# In the training loop:
out = model(train_data)
loss = torch.nn.MSELoss()(out.view(-1), train_data.y.view(-1))


NameError: name 'model' is not defined

In [ ]:
def read_and_process(directory):
    files = os.listdir(directory)
    df_list = []
    for file in files:
        if file.endswith('.csv'):
            df = pd.read_csv(os.path.join(directory, file))
            df = df.drop(['timestamp'], axis=1)
            df = df.dropna()
            df_list.append(df)
    df_combined = pd.concat(df_list, ignore_index=True)
  
    return df_combined

In [ ]:
data_dir = 'dms_data'
# Define directory paths
train_data_path = data_dir + '/train'
test_data_path = data_dir + '/test'

# Read and process data
train = read_and_process(train_data_path)
test = read_and_process(test_data_path)
features = ['m_speed', 'm_speed_var_480', 'm_speed_stddev_480', 'm_acceleration',
    'm_acceleration_var_480', 'm_acceleration_stddev_480', 'm_jerk',
    'm_jerk_var_480', 'm_jerk_stddev_480']
X_train, y_train, X_test, y_test = train[features], train['oss'], test[features], test['oss']

# ここから

# elif model_type == "gnn":
def trainGNN(X_train,y_train,X_test,y_test):
    train_data = transform_to_graph_data(X_train, y_train)
    test_data = transform_to_graph_data(X_test, y_test)

    # Convert y_train from pandas Series to PyTorch tensor and reshape it
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.float).view(-1, 1)

    # Define the GNN model
    model = SimpleGNN(num_features=X_train.shape[1])

    # Train the GNN model
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    for epoch in range(10):
        model.train()
        optimizer.zero_grad()
        out = model(train_data)

        loss = torch.nn.MSELoss()(out, y_train_tensor)
        loss.backward()
        optimizer.step()

    # Evaluate the GNN model
    model.eval()
    preds = model(test_data)
    rmse = np.sqrt(mean_squared_error(y_test, preds.detach().numpy()))

    return rmse
# ここまで
def compute_statistics(arr):
    median = np.median(arr)
    mean = np.mean(arr)
    variance = np.var(arr)
    std_dev = np.std(arr)
    range_val = (np.min(arr), np.max(arr))
    
    return {
        'Median': median,
        'Mean': mean,
        'Variance': variance,
        'Standard Deviation': std_dev,
        'Range': range_val
    }
rmses = []
for i in range(10):
    rmses.append(trainGNN(X_train,y_train,X_test,y_test))
print(compute_statistics(rmses))

5.895394481179766


In [ ]:
data_dir = 'dms_data'
# Define directory paths
train_data_path = data_dir + '/train'
test_data_path = data_dir + '/test'

# Read and process data
train = read_and_process(train_data_path)
test = read_and_process(test_data_path)

train.head()

,m_speed,m_speed_var_480,m_speed_stddev_480,m_acceleration,m_acceleration_var_480,m_acceleration_stddev_480,m_jerk,m_jerk_var_480,m_jerk_stddev_480,m_steering,...,BrakeInput,BrakeInput_var_480,BrakeInput_stddev_480,realtime steering entropy_1100,realtime steering entropy_1100_var_480,realtime steering entropy_1100_stddev_480,perclos,mean_ec_out,oss,Sleepiness
0,69.2,62.888737,7.930242,0.0736,0.003503,0.059185,0.0015,0.000020,0.004417,0.000987,...,0.0,0.000177,0.013288,0.000427,0.0,0.000056,0.294543,1.873714,2.384637,4.0
1,63.4,72.544368,8.517298,-0.1315,0.004115,0.064149,0.0003,0.000020,0.004455,-0.000804,...,0.0,0.000177,0.013302,0.000895,0.0,0.000207,0.228587,0.000009,2.010695,4.0
2,101.0,87.896412,9.375309,0.0644,0.004418,0.066471,-0.0001,0.000020,0.004455,0.000466,...,0.0,0.000177,0.013315,0.000504,0.0,0.000222,0.140712,0.081118,2.135257,4.0
3,86.6,86.736607,9.313249,0.0225,0.004566,0.067569,0.0000,0.000020,0.004480,-0.000063,...,0.0,0.000178,0.013328,0.000000,0.0,0.000224,0.134744,0.000583,2.658373,4.0
4,52.4,80.914104,8.995227,0.0184,0.004323,0.065747,0.0000,0.000017,0.004174,-0.000401,...,0.0,0.000178,0.013338,0.000295,0.0,0.000230,0.133779,0.106500,2.471206,4.0


In [ ]:
train.dtypes

m_speed                                      float64
m_speed_var_480                              float64
m_speed_stddev_480                           float64
m_acceleration                               float64
m_acceleration_var_480                       float64
m_acceleration_stddev_480                    float64
m_jerk                                       float64
m_jerk_var_480                               float64
m_jerk_stddev_480                            float64
m_steering                                   float64
m_steering_var_480                           float64
m_steering_stddev_480                        float64
AccelInput                                   float64
AccelInput_var_480                           float64
AccelInput_stddev_480                        float64
BrakeInput                                   float64
BrakeInput_var_480                           float64
BrakeInput_stddev_480                        float64
realtime steering entropy_1100               f

In [ ]:
train['oss'].value_counts()

oss
2.666667    189
2.333333    138
3.000000    107
3.333333     61
5.000000     53
           ... 
3.666661      1
3.931794      1
3.931771      1
3.598381      1
4.521394      1
Name: count, Length: 1088, dtype: int64